# Data preparation and data cleaning (1)

In [1]:
import numpy as np
import pandas as pd

In [2]:
# Load raw train and test datasets.
train_raw = pd.read_csv("train_raw.csv", low_memory=False)
test_raw = pd.read_csv("test_raw.csv", low_memory=False)

In [3]:
train_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300000 entries, 0 to 299999
Columns: 333 entries, CUSTOMER_ID to RESPONDERS
dtypes: float64(236), int64(4), object(93)
memory usage: 762.2+ MB


From the train_raw.info(), we can know:
1) The train dataset has 300,000 rows.
2) The train dataset has 333 columns. The 1st column is CUSTOMER_ID, and the last column is RESPONDERS, the target variable. The left 331 columns are variables.
3) The dtype of 236 columns is float64, the dtype of 4 columns is int64, and the dtype of 93 columns is object.

In [4]:
test_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Columns: 332 entries, CUSTOMER_ID to SCRUB_EMI
dtypes: float64(238), int64(4), object(90)
memory usage: 506.6+ MB


From the test_raw.info(), we can know:
1) The test dataset has 200,000 rows.
2) The test dataset has 332 columns. The 1st column is CUSTOMER_ID. The left 331 columns are variables.
3) The dtype of 238 columns is float64, the dtype of 4 columns is int64, and the dtype of 90 columns is object.

In [5]:
# Examine the number of values in every column of the train dataset.

val_counts = []
for i in range(0, train_raw.shape[1]):
    val_counts.append(train_raw.iloc[:, i].count())
val_counts_sorted = sorted(val_counts)
print(val_counts_sorted[:20])
print()
print(val_counts_sorted[-20:])

[0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 2, 3, 6, 12, 14, 15, 16, 18, 18, 18]

[300000, 300000, 300000, 300000, 300000, 300000, 300000, 300000, 300000, 300000, 300000, 300000, 300000, 300000, 300000, 300000, 300000, 300000, 300000, 300000]


Many columns of the train dataset have a very small amount of values. The test set has the same problem.
Next, I'm going to remove those columns that have a small amount of values from both train and test sets.

In [6]:
# To generate lists of selected columns containing a large number of values in the train set.
# Here, columns containing >= (len(train.shape[0]))/3, that is 300,000/3=100,000, values are chosen.
# This list will be used when generating new datasets.
# A sub list of columns with object dtype is also generated.

all_train_cols = list(train_raw)
train_cols = []    # List of columns in train set to be chosen.
train_cols_obj = []    # A sub list of columns with object dtype.
for col in all_train_cols:
    if (train_raw[col].count()) >= ((train_raw.shape[0])/3):
        train_cols.append(col)
        if train_raw[col].dtype == "object":
            train_cols_obj.append(col)

In [7]:
# To check the number of unique values of selected object columns

num_obj_var = []
for col in train_cols_obj:
    df = train_raw.loc[:, [col]]
    df_array = df.values
    set_ = set(df_array[:,0])
    num_obj_var.append((len(set_), col))
print(sorted(num_obj_var))

[(1, 'OCCUP_ALL_NEW'), (2, 'ACT_TYPE'), (2, 'CARD_AUTOMOBILE_MON_01'), (2, 'CARD_ENTMNT_MON_01'), (2, 'CARD_HOBBY_MON_01'), (2, 'CARD_HOME_DECOR_MON_01'), (2, 'CARD_HOTEL_MON_01'), (2, 'CARD_JEWELLERY_MON_01'), (2, 'CARD_MEDICAL_MON_01'), (2, 'CARD_PRSNL_CARE_MON_01'), (2, 'CARD_RESTAURANT_MON_01'), (2, 'CARD_TRAVEL_MON_01'), (2, 'CC_ACTIVE'), (2, 'CC_HOLD'), (2, 'CREDIT_ACT'), (2, 'DC_ACTIVE'), (2, 'DC_HOLD'), (2, 'DEBIT_ACT'), (2, 'FINANCE_MON_01'), (2, 'INMON_01KET_MON_01'), (2, 'JOBS_MON_01'), (2, 'LIFESTYLE_MON_01'), (2, 'RESPONDERS'), (2, 'TOP14_CITY'), (2, 'TOP9_CITY'), (2, 'TRAVEL_MON_01'), (3, 'CHANNEL_CLICK_DISP'), (3, 'GENDER'), (3, 'TOP_CORP_TAG'), (5, 'LEGAL_ENTITY'), (11, 'DESIGNATION_FINAL'), (20, 'NEFT_CC_CATEGORY'), (20, 'NEFT_DC_CATEGORY'), (21, 'IMPS_CC_CATEGORY_MON_01'), (21, 'TPT_CC_CATEGORY_MON_01'), (21, 'TPT_DC_CATEGORY_MON_01'), (141, 'PA_PQ_TAG'), (27857, 'ZIP_CODE_FINAL')]


The variable "OCCUP_ALL_NEW" only has one unique value in both train and test datasets. It can be removed. The variable "ZIP_CODE_FINAL" has the most unique values (27857) and the variable "PA_PQ_TAG" also has many unique values (141). I'll check the values in the two variables, find the pattern of the values, and transform the object values to numeral values.

In [8]:
# Remove OCCUP_ALL_NEW from the list of selected columns.
train_cols.remove("OCCUP_ALL_NEW")

In [9]:
len(train_cols)

88

In [10]:
# Generate new train and test sets according to the list of selected column.
# Also generate a subset of Customer_ID for test

# Generate new train set and output as csv file. The new train set has 86 features and the response variable
train = train_raw[train_cols[1:]]
train.to_csv("train1.csv", index=None)

# Generate a Customer_ID set for test and output as csv file. It will be used when reporting the prediction result.
test_customer_id = test_raw[train_cols[0]]
test_customer_id.to_csv("test_customer_id.csv", index=None)

# Generate new test set and output as csv file. The new test set has 86 features only.
test = test_raw[train_cols[1:-1]]
test.to_csv("test1.csv", index=None)

Now, we can stop here and take a break.